# Estimación de Modelo Probit

### Conjunto de datos: participación laboral femenina

* En este ejercicio trabajamos en el conjunto de datos de Mroz sobre la participación laboral femenina con 8 variables.

* Los datos cubren una muestra de 753 mujeres blancas casadas de entre 30 y 60 años en 1975. 

* La fuente original de estos datos es Mroz, T.A. (1987). “The sensitivity of an empirical model of married women’s hours of work to economic and statistical assumptions.” Econometrica 55, 765-799.

* La descripción de las variables se puede encontrar a continuación:

lfp: Labor-force participation of the married white woman (Categorical: 0/1)

k5: Number of children younger than 6 years old	(Entero positivo)

k618: Number of children aged 6-18 (Entero positivo)

age: Age in years (Entero positivo)

wc: Wife's college attendance (Categorical: 0/1)

hc: Husband's college attendance (Categorical: 0/1)

lwg: Log expected wage rate for women in the labor force (Numerical)

inc: Family income without the wife's income (Numerical)

# 1. Importación de bibliotecas y datos

In [ ]:
# Dependecies:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from statsmodels.discrete.discrete_model import Probit

#
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Importación de datos
data = pd.read_csv('Mroz.csv')
data.head()

In [ ]:
# Limpieza de datos:
#
data["lfp_d"] = 0
data.loc[data.lfp == "yes", "lfp_d"] = 1

#
data["wc_d"] = 0
data.loc[data.wc == "yes", "wc_d"] = 1

#
data["hc_d"] = 0
data.loc[data.hc == "yes", "hc_d"] = 1

#
data

In [ ]:
# Estadísticas descriptivas:

data.describe()

# 2. Estimación del modelo Probit

In [ ]:
# Definimos variables dependiente e independientes:

Y = data['lfp_d']
X = data[['k5', 'k618', 'age', 'wc_d', 'hc_d', 'lwg', 'inc']]
X = sm.add_constant(X)

In [ ]:
# Estimación:

model = Probit(Y, X.astype(float))
probit_model = model.fit()
print(probit_model.summary())

# 3. Efectos marginales

Algunas caracterísricas de los datos:

In [ ]:
# Tablas de frecuencias por categoria para hc: Husband's college attendance (Categorical: 0/1)

pd.crosstab(data["lfp"], data["hc"], margins = True)

\begin{eqnarray*}
    EMg_j & = & P(y = 1 | \mathbf{x}_i, x_j = 1) - P(y = 1 | \mathbf{x}_i, x_j = 0) \\
    & = & G(\mathbf{x}_i \boldsymbol{\beta} | x_j = 1) - G(\mathbf{x}_i \boldsymbol{\beta} | x_j = 0) \\
    & = & G(\beta_1 + x_{2} \beta_2 + \ldots + \beta_j + \ldots + x_{K} \beta_K) \\
    &  & - G(\beta_1 + x_{2} \beta_2 + \ldots + 0 + \ldots + x_{K} \beta_K)
\end{eqnarray*}

In [ ]:
# hc: Husband's college attendance = 0
hc_data0 = np.column_stack((
                            1, np.mean(data["k5"]), np.mean(data["k618"]), np.mean(data["age"]),
                            np.mean(data["wc_d"]), 0, np.mean(data["lwg"]), np.mean(data["inc"])
                           ))

hc_data0

In [ ]:
# hc: Husband's college attendance = 1
hc_data1 = np.column_stack((
                            1, np.mean(data["k5"]), np.mean(data["k618"]), np.mean(data["age"]),
                            np.mean(data["wc_d"]), 1, np.mean(data["lwg"]), np.mean(data["inc"])
                           ))

hc_data1

In [ ]:
# Estimación para el primero: D = 0
print(probit_model.predict(hc_data0))

In [ ]:
# Estimación para el segundo: D = 1
print(probit_model.predict(hc_data1))

In [ ]:
# Efecto Marginal:
probit_model.predict(hc_data1) - probit_model.predict(hc_data0)

In [ ]:
# Tablas de frecuencias por categoria para wc: Wife's college attendance (Categorical: 0/1)

pd.crosstab(data["lfp"], data["wc"], margins = True)

In [ ]:
# wc: Wife's college attendance = 0
wc_data0 = np.column_stack((1, np.mean(data["k5"]), np.mean(data["k618"]), np.mean(data["age"]), 
                            0, np.mean(data["hc_d"]), np.mean(data["lwg"]), np.mean(data["inc"])))

wc_data0

In [ ]:
# wc: Wife's college attendance = 0
wc_data1 = np.column_stack((1, np.mean(data["k5"]), np.mean(data["k618"]), np.mean(data["age"]), 
                            1, np.mean(data["hc_d"]), np.mean(data["lwg"]), np.mean(data["inc"])))

wc_data1

In [ ]:
print(probit_model.predict(wc_data0)), print(probit_model.predict(wc_data1))

In [ ]:
probit_model.predict(wc_data1) - probit_model.predict(wc_data0)

In [ ]:
# Tablas de frecuencias por categoria para k5: Number of children younger than 6 years old (Entero positivo)

pd.crosstab(data["lfp"], data["k5"], margins = True)

\begin{eqnarray*}
    EMg_j & = & P(y = 1 | \mathbf{x}_i, x_j = 1) - P(y = 1 | \mathbf{x}_i, x_j = 0) \\
    & = & G(\mathbf{x}_i \boldsymbol{\beta} | x_j = C + 1) - G(\mathbf{x}_i \boldsymbol{\beta} | x_j = C) \\
\end{eqnarray*}

In [ ]:
k5_data = np.column_stack(( np.repeat(1,4), (0,1,2,3),
                           np.repeat(np.mean(data["k618"]), 4), np.repeat(np.mean(data["age"]),4),
                           np.repeat(np.mean(data["wc_d"]),4), np.repeat(np.mean(data["hc_d"]),4),
                           np.repeat(np.mean(data["lwg"]),4), np.repeat(np.mean(data["inc"]),4) ))

k5_data

In [ ]:
print(probit_model.predict(k5_data))

Sin importar el modelo que estemos ocupando, la forma de interpretar el modelo es mediante el efecto marginal, cuando $x_j$ sea una variable continua:
\begin{equation*}
    EMg_j = \frac{\partial}{\partial x_j} P(y = 1 | \mathbf{x}_i) = \frac{\partial}{\partial x_j} G(\mathbf{x}_i \boldsymbol{\beta}) = g(\mathbf{x}_i \boldsymbol{\beta}) \beta_j
\end{equation*}

In [ ]:
#
mfx = probit_model.get_margeff()

print(mfx.summary())